In [175]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt

from tlo import Date, Simulation, logging
from tlo.analysis.utils import parse_log_file
from tlo.methods import (
    demography,
    enhanced_lifestyle,
    healthburden,
    healthseekingbehaviour,
    healthsystem,
    rti,
    simplified_births,
    symptommanager,
)
from tlo.analysis.utils import summarize,extract_results

In [178]:
from tlo.analysis.utils import create_pickles_locally, parse_log_file

# File paths


# Parse the log file

folder = Path("/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z")
# # get the pickled files if not generated at the batch run
create_pickles_locally(scenario_output_dir = folder, compressed_file_name_prefix='longterm_trends_all_diseases__')

Opening /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z/0/9/longterm_trends_all_diseases__2024-11-19T091939.log
Processing log file /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z/0/9/longterm_trends_all_diseases__2024-11-19T091939.log
Writing module-specific log files to /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z/0/9
Finished writing module-specific log files.
 - Writing tlo.simulation.pickle
Opening /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z/0/0/longterm_trends_all_diseases__2024-11-19T091958.log
Processing log file /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/longterm_trends_all_diseases-2024-11-19T091704Z/0/0/longterm_trends_all_diseases__2024-11-19T091958.log
Writing module-specific log files to /Users/rem76/Pycharm

In [176]:


pop_model = summarize(extract_results(
    folder,
    module="tlo.methods.demography",
    key="population",
    column="total",
    index="date",
    do_scaling=True
),
    collapse_columns=True
)

mean_li_urban_immediate_death = summarize(extract_results(
    folder,
    module="tlo.methods.demography.detail",
    key="properties_of_deceased_persons",
    custom_generate_series=(
        lambda df: df.loc[(df['li_urban']) &
                          (df['cause_of_death'].str.contains('RTI_imm_death'))].assign(
            year=df['date'].dt.year).groupby(['year'])['year'].count()
    ),
    do_scaling=True
))

mean_li_urban_immediate_death = mean_li_urban_immediate_death.reset_index()
mean_li_urban_immediate_death.columns = ['year', 'lower', 'mean', 'upper']

mean_li_urban_non_immediate_death = summarize(extract_results(
    folder,
    module="tlo.methods.demography.detail",
    key="properties_of_deceased_persons",
    custom_generate_series=(
        lambda df: df.loc[(df['li_urban']) &
                          (df['cause_of_death'].str.contains('RTI_')) &
                          (~df['cause_of_death'].str.contains('RTI_imm_death'))].assign(
            year=df['date'].dt.year).groupby(['year'])['year'].count()
    ),
    do_scaling=True
))

mean_li_urban_non_immediate_death = mean_li_urban_non_immediate_death.reset_index()
mean_li_urban_non_immediate_death.columns = ['year', 'lower', 'mean', 'upper']

pop_model = pop_model.reset_index()
pop_model.columns = ['year', 'lower', 'mean', 'upper']

mean_li_all_immediate_death = summarize(extract_results(
    folder,
    module="tlo.methods.demography.detail",
    key="properties_of_deceased_persons",
    custom_generate_series=(
        lambda df: df.loc[
            (df['cause_of_death'].str.contains('RTI_imm_death'))].assign(
            year=df['date'].dt.year).groupby(['year'])['year'].count()
    ),
    do_scaling=True
))

mean_li_all_immediate_death = mean_li_all_immediate_death.reset_index()
mean_li_all_immediate_death.columns = ['year', 'lower', 'mean', 'upper']

mean_li_all_non_immediate_death = summarize(extract_results(
    folder,
    module="tlo.methods.demography.detail",
    key="properties_of_deceased_persons",
    custom_generate_series=(
        lambda df: df.loc[
            (df['cause_of_death'].str.contains('RTI_')) &
            (~df['cause_of_death'].str.contains('RTI_imm_death'))].assign(
            year=df['date'].dt.year).groupby(['year'])['year'].count()
    ),
    do_scaling=True
))

mean_li_all_non_immediate_death = mean_li_all_non_immediate_death.reset_index()
mean_li_all_non_immediate_death.columns = ['year', 'lower', 'mean', 'upper']
mean_li_all_non_immediate_death = mean_li_all_non_immediate_death.sort_values(by='year', ascending=True)

mean_li_all_death_with_med = summarize(extract_results(
    folder,
    module="tlo.methods.demography.detail",
    key="properties_of_deceased_persons",
    custom_generate_series=(
        lambda df: df.loc[(df['cause_of_death'].str.contains('RTI_death_with_med'))].assign(
            year=df['date'].dt.year).groupby(['year'])['year'].count()
    ),
    do_scaling=True
))

mean_li_all_death_with_med = mean_li_all_death_with_med.reset_index()
mean_li_all_death_with_med.columns = ['year', 'lower', 'mean', 'upper']
mean_li_all_death_with_med = mean_li_all_death_with_med.sort_values(by='year', ascending=True)

plt.plot(range(len(mean_li_all_non_immediate_death['year'])),
         mean_li_all_non_immediate_death['mean'] / pop_model['mean'] * 100000, color='blue')
plt.title("mean_li_all_immediate_death")
plt.ylabel("Deaths due to RTI per 100,000")
plt.xlabel("Year")
plt.show()

mean_li_all_immediate_death = mean_li_all_immediate_death.sort_values(by='year', ascending=True)
plt.plot(range(len(mean_li_all_immediate_death['year'])),
         mean_li_all_immediate_death['mean'] / pop_model['mean'] * 100000, color='red')
plt.title("mean_li_all_immediate_death")
plt.ylabel("Deaths due to RTI per 100,000")
plt.xlabel("Year")
plt.show()

mean_li_all_death_with_med = mean_li_all_death_with_med.sort_values(by='year', ascending=True)
plt.plot(range(len(mean_li_all_death_with_med['year'])),
         mean_li_all_death_with_med['mean'] / pop_model['mean'][0:len(mean_li_all_death_with_med['mean'])] * 100000,
         color='blue')
plt.title("mean_li_all_death_with_med")
plt.ylabel("Deaths due to RTI per 100,000")
plt.xlabel("Year")
plt.show()


ValueError: All objects passed were None